# 4.1 - Procesos asíncronos


![async](images/async.png)




**[Documentación](https://docs.python.org/3/library/asyncio.html)**


**asyncio** es una biblioteca para escribir código [concurrente](https://es.wikipedia.org/wiki/Concurrencia_(inform%C3%A1tica)) utilizando la sintaxis async/await. Se utiliza como base en múltiples frameworks asíncronos de Python y provee un alto rendimiento en redes y servidores web, bibliotecas de conexión de base de datos, colas de tareas distribuidas, etc.

Suele encajar perfectamente para operaciones con límite de E/S y código de red estructurado de alto nivel. Además provee un conjunto de APIs de alto nivel para:

+ ejecutar corutinas de Python de manera concurrente y tener control total sobre su ejecución

+ realizar redes E/S y comunicación entre procesos(IPC)

+ controlar subprocesos

+ distribuir tareas a través de colas

+ sincronizar código concurrente

Adicionalmente, existen APIs de bajo nivel para desarrolladores de bibliotecas y frameworks para:

+ crear y administrar bucles de eventos, los cuales proveen APIs asíncronas para redes, ejecutando subprocesos, gestionando señales del sistema operativo, etc..

+ implementar protocolos eficientes utilizando transportes

+ bibliotecas puente basadas en retrollamadas y código con sintaxis async/wait

$$$$

Nosotros nos enfocaremos en el uso de bucles de eventos para la extracción de datos de la web.


### Ejemplo ESPN

Volvamos al ejemplo de scrapeo de la págine de ESPN. Vamos a realizar múltiples requests para obtener los datos de todos los equipos.


https://www.espn.com/soccer/competitions

In [1]:
import asyncio


from selenium import webdriver
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import Select

import time

import pandas as pd

from tqdm.notebook import tqdm

#from webdriver_manager.chrome import ChromeDriverManager
#PATH=ChromeDriverManager().install()

PATH='driver/chromedriver'

import warnings
warnings.filterwarnings('ignore')

In [2]:
url = 'https://www.espn.com/soccer/competitions'

In [3]:
driver = webdriver.Chrome(PATH)

driver.get(url)

In [4]:
# cookies

aceptar = driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')

aceptar.click()

In [5]:
# selecciona equipos laliga

equipos = driver.find_element(By.XPATH, '//*[@id="fittPageContainer"]/div[3]/div/div/div/div[2]/div[2]/div/div[5]/div/section/div/div/span[2]/a')

equipos.click()

In [6]:
stats = driver.find_elements(By.CSS_SELECTOR, 'a.AnchorLink')   # tag.clase

stats[12].get_attribute('href')

'https://www.espn.com/mlb/boxscore/_/gameId/401472644'

In [13]:
[e.text for e in stats][70:80]

['Home',
 "Women's World Cup",
 'Scores',
 'Schedule',
 'Transfers',
 'More',
 '',
 'Alavés',
 'Fixtures',
 'Stats']

In [14]:
team_stats = []

for e in stats:
    
    try:
        
        link = e.get_attribute('href')
        
        if 'soccer/team/stats' in link:
            team_stats.append(link)
        else:
            continue
    except:
        continue
        
driver.quit()


team_stats[:10]

['https://www.espn.com/soccer/team/stats/_/id/96/alaves',
 'https://www.espn.com/soccer/team/stats/_/id/6832/almeria',
 'https://www.espn.com/soccer/team/stats/_/id/93/athletic-club',
 'https://www.espn.com/soccer/team/stats/_/id/1068/atletico-madrid',
 'https://www.espn.com/soccer/team/stats/_/id/83/barcelona',
 'https://www.espn.com/soccer/team/stats/_/id/85/celta-vigo',
 'https://www.espn.com/soccer/team/stats/_/id/3842/cadiz',
 'https://www.espn.com/soccer/team/stats/_/id/2922/getafe',
 'https://www.espn.com/soccer/team/stats/_/id/9812/girona',
 'https://www.espn.com/soccer/team/stats/_/id/3747/granada']

In [16]:
team_stats

['https://www.espn.com/soccer/team/stats/_/id/96/alaves',
 'https://www.espn.com/soccer/team/stats/_/id/6832/almeria',
 'https://www.espn.com/soccer/team/stats/_/id/93/athletic-club',
 'https://www.espn.com/soccer/team/stats/_/id/1068/atletico-madrid',
 'https://www.espn.com/soccer/team/stats/_/id/83/barcelona',
 'https://www.espn.com/soccer/team/stats/_/id/85/celta-vigo',
 'https://www.espn.com/soccer/team/stats/_/id/3842/cadiz',
 'https://www.espn.com/soccer/team/stats/_/id/2922/getafe',
 'https://www.espn.com/soccer/team/stats/_/id/9812/girona',
 'https://www.espn.com/soccer/team/stats/_/id/3747/granada',
 'https://www.espn.com/soccer/team/stats/_/id/98/las-palmas',
 'https://www.espn.com/soccer/team/stats/_/id/84/mallorca',
 'https://www.espn.com/soccer/team/stats/_/id/97/osasuna',
 'https://www.espn.com/soccer/team/stats/_/id/101/rayo-vallecano',
 'https://www.espn.com/soccer/team/stats/_/id/244/real-betis',
 'https://www.espn.com/soccer/team/stats/_/id/86/real-madrid',
 'https://

In [15]:
len(team_stats)

20

**Extracción asincrónica**

In [19]:
help(asyncio.get_event_loop().run_in_executor)

Help on method run_in_executor in module asyncio.base_events:

run_in_executor(executor, func, *args) method of asyncio.unix_events._UnixSelectorEventLoop instance



In [20]:
def asincrono(funcion):
    
    def eventos(*args, **kwargs):
        return asyncio.get_event_loop().run_in_executor(None, funcion, *args, **kwargs)
    
    return eventos

In [22]:
DATOS=[]

CABECERAS=[]

In [23]:
@asincrono
def extraer(url):
    
    global DATOS, CABECERAS, PATH
    
    
    # iniciar el driver
    driver=webdriver.Chrome(PATH)
    driver.get(url)
    
    time.sleep(2)
    
    # acepta cookies
    try:
        aceptar=driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
        aceptar.click()

        time.sleep(2)  # espera 2 secs
    except:
        time.sleep(1) # espera 1 secs
        
    # dropdown
    dropdown = driver.find_element(By.XPATH, '//*[@id="fittPageContainer"]/div[2]/div[5]/div/div/section/div/div[4]/select[1]')
    select = Select(dropdown)
    select.select_by_visible_text('2022-23')
    
    time.sleep(1)
    
    # disciplina
    dis=driver.find_element(By.XPATH,'//*[@id="fittPageContainer"]/div[2]/div[5]/div/div[1]/section/div/div[2]/nav/ul/li[2]/a')
    dis.click()
    
    time.sleep(2)

    tabla=driver.find_element(By.TAG_NAME,'tbody')

    filas=tabla.find_elements(By.TAG_NAME, 'tr')

    data=[]
    
    for f in filas:

        elementos=f.find_elements(By.TAG_NAME, 'td') 

        tmp=[]

        for e in elementos:

            tmp.append(e.text)

        tmp.append(url.split('/')[-1])  # añade el nombre del equipo
        
        data.append(tmp)
        
        
    cabeceras=driver.find_element(By.TAG_NAME, 'thead')

    cabeceras=[c.text for c in cabeceras.find_elements(By.TAG_NAME, 'th')]+['TEAM']


    DATOS+=data

    CABECERAS=cabeceras
    
    

In [24]:
%%time

for url in tqdm(team_stats[:10]):
    
    res = extraer(url)
    display(res)

  0%|          | 0/10 [00:00<?, ?it/s]

<Future pending cb=[_chain_future.<locals>._call_check_cancel() at /opt/homebrew/Caskroom/miniconda/base/envs/clase/lib/python3.9/asyncio/futures.py:384]>

<Future pending cb=[_chain_future.<locals>._call_check_cancel() at /opt/homebrew/Caskroom/miniconda/base/envs/clase/lib/python3.9/asyncio/futures.py:384]>

<Future pending cb=[_chain_future.<locals>._call_check_cancel() at /opt/homebrew/Caskroom/miniconda/base/envs/clase/lib/python3.9/asyncio/futures.py:384]>

<Future pending cb=[_chain_future.<locals>._call_check_cancel() at /opt/homebrew/Caskroom/miniconda/base/envs/clase/lib/python3.9/asyncio/futures.py:384]>

...

<Future pending cb=[_chain_future.<locals>._call_check_cancel() at /opt/homebrew/Caskroom/miniconda/base/envs/clase/lib/python3.9/asyncio/futures.py:384]>

<Future pending cb=[_chain_future.<locals>._call_check_cancel() at /opt/homebrew/Caskroom/miniconda/base/envs/clase/lib/python3.9/asyncio/futures.py:384]>

<Future pending cb=[_chain_future.<locals>._call_check_cancel() at /opt/homebrew/Caskroom/miniconda/base/envs/clase/lib/python3.9/asyncio/futures.py:384]>

<Future pending cb=[_chain_future.<locals>._call_check_cancel() at /opt/homebrew/Caskroom/miniconda/base/envs/clase/lib/python3.9/asyncio/futures.py:384]>

CPU times: user 38.2 ms, sys: 60.5 ms, total: 98.7 ms
Wall time: 97.3 ms


In [ ]:
await   # esperar hasta acabar

In [29]:
df=pd.DataFrame(DATOS, columns=CABECERAS)

df.shape

(193, 7)

In [30]:
df.head()

,RK,NAME,P,YC,RC,PTS,TEAM
0,1,Renato Tapia,28,7,1,10,celta-vigo
1,2,Gabriel Veiga,36,6,1,9,celta-vigo
2,,Hugo Mallo,26,6,1,9,celta-vigo
3,4,Unai Núñez,36,8,0,8,celta-vigo
4,,Joseph Aidoo,35,5,1,8,celta-vigo


In [31]:
df.TEAM.unique()

array(['celta-vigo', 'girona', 'cadiz', 'almeria', 'atletico-madrid',
       'athletic-club', 'barcelona', 'getafe'], dtype=object)